# SI 618 Classification II (Evaluation)

Today we're going to focus on model evaluation.  The first part of this notebook is a copy of one we covered earlier.

In [ ]:
import seaborn as sns
import pandas as pd

In [ ]:
data = sns.load_dataset('iris')
data.groupby('species').mean()

In [ ]:
from sklearn.model_selection import train_test_split
# "target" contains the column name of the classification labels
target = "species"

X = data.drop(target,1)
y = data[target]

seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix

## Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

model = GaussianNB()
model.fit(X_train, y_train)
# make predictions for test data
y_pred = model.predict(X_test)

$\text{accuracy} = \frac{\text{True Positives + True Negatives}}{\text{All Samples}}$

$\text{precision} = \frac{\text{True Positives}}{\text{True Positives + False Positives}}$

$\text{true positive rate} = \text{recall} = \text{sensitivity} = \frac{\text{True Positives}}{\text{True Positives + False Negatives}}$

$\text{F1} = \frac{2 \times (\text{Precision} \times \text{Recall})}{\text{Precision + Recall}}$

$ \text{false positive rate} = \text{fall-out} = \frac{\text{False Positives}}{\text{False Positives + True Negatives}}$

$ \text{specificity} = \frac{\text{True Negatives}}{\text{True Negatives + False Positives}}$

$ \text{false positive rate} = 1 - \text{specificity}$

In [ ]:
# evaluate predictions
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='liblinear',multi_class="auto")
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

## Support Vector Machines

In [ ]:
from sklearn import svm
model = svm.SVC(gamma="scale",kernel="rbf")
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


## Decision Trees

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
model = DecisionTreeClassifier(
    random_state=3, 
    criterion='entropy',
    splitter='best', 
    max_depth=5, 
    min_samples_split=2)

model.fit(X_train,y_train)

y_pred = model.predict(X_test)

# evaluate predictions
accuracy = model.score(X_test, y_test)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree

In [ ]:
# based on https://mljar.com/blog/visualize-decision-tree/
fig = plt.figure(figsize=(25,20))
# we use '_' to capture the return from plot_tree(...) and never do anything with it
_ = plot_tree(model, 
                   feature_names=X.columns,  # ordered list of features
                   class_names=y.unique(), # unique() is handy here due to numerical sorting of classes
                   filled=True)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

## Random Forests

In [ ]:
seed = 42

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

model = RandomForestClassifier(
    n_estimators=100,
    random_state=seed, 
    criterion='entropy',
    max_depth=None, 
    min_samples_split=2)

model.fit(X_train,y_train)

y_pred = model.predict(X_test)

# evaluate predictions
accuracy = model.score(X_test, y_test)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
# Check feature importance
feat_importance = model.feature_importances_
pd.DataFrame({'Feature Importance':feat_importance},
            index=list(X)).plot(kind='barh')

##  Cross-validation




### Tuning the model
The model paramters are called _hyperparameters_.  Setting them to appropriate values and running the model experiment again can be tedious, so we have a technique called cross-validation to help with this.

From http://scikit-learn.org/stable/modules/cross_validation.html:

> When evaluating different settings (“hyperparameters”) for estimators, such as the C setting that must be manually set for an SVM, there is still a risk of overfitting on the test set because the parameters can be tweaked until the estimator performs optimally. This way, knowledge about the test set can “leak” into the model and evaluation metrics no longer report on generalization performance. To solve this problem, yet another part of the dataset can be held out as a so-called “validation set”: training proceeds on the training set, after which evaluation is done on the validation set, and when the experiment seems to be successful, final evaluation can be done on the test set.

> However, by partitioning the available data into three sets, we drastically reduce the number of samples which can be used for learning the model, and the results can depend on a particular random choice for the pair of (train, validation) sets.

> A solution to this problem is a procedure called cross-validation (CV for short). A test set should still be held out for final evaluation, but the validation set is no longer needed when doing CV. In the basic approach, called k-fold CV, the training set is split into k smaller sets (other approaches are described below, but generally follow the same principles). The following procedure is followed for each of the k “folds”:

> A model is trained using  of the folds as training data;
the resulting model is validated on the remaining part of the data (i.e., it is used as a test set to compute a performance measure such as accuracy).
The performance measure reported by k-fold cross-validation is then the average of the values computed in the loop. This approach can be computationally expensive, but does not waste too much data (as is the case when fixing an arbitrary validation set), which is a major advantage in problems such as inverse inference where the number of samples is very small.

The mean score and the 95% confidence interval (2 x the standard deviation) of the score estimate are hence given by:


In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model,X,y, cv=5 , n_jobs=-1)
print("Accuracy: %0.2f%% (+/- %0.2f)" % (scores.mean()*100, scores.std() * 2*100))

But let's say we want to fiddle with the hyperparameters.

To do this, we first set up a parameter grid with the lists of paramters we want to try:

In [ ]:
param_grid = {
                 'n_estimators': [5, 10, 15, 20, 25, 30, 35, 40, 50],
                 'max_depth': [2, 5, 7, 9],
             }

We now run the cross-validation on the classifier. Note: this will take a while (why?).

In [ ]:
from sklearn.model_selection import GridSearchCV

grid_clf = GridSearchCV(model, param_grid, cv=10, n_jobs=-1)
grid_clf.fit(X,y)

In [ ]:
grid_clf.best_estimator_

In [ ]:
grid_clf.best_params_

In [ ]:
# Details, which can be imported into a pandas dataframe:
results = pd.DataFrame(grid_clf.cv_results_)
results.head()

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model,X,y, cv=10)
print("Accuracy: %0.2f%% (+/- %0.2f)" % (scores.mean()*100, scores.std() * 2*100))

## Task
Recall the `penguins` dataset from Day 14.  Choosing from Naive Bayes, Logistic Regression, Support Vector Machines, Decision Trees and Random Forests, find the "best" classifier to classify penguin species.  Note that you can set up your code to run one classifier, and then change only the classified specification and keep the rest of the code the same.

In [ ]:
penguins = sns.load_dataset('penguins')
penguins.dropna(inplace=True)

In [ ]:
penguins.species.unique()

In [ ]:
penguins['species'].value_counts()

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in sss.split(penguins, penguins['species']):
    penguins_train_set = penguins.loc[penguins.index.intersection(train_index)]
    penguins_test_set = penguins.loc[penguins.index.intersection(test_index)]

In [ ]:
penguins_train_set.shape,penguins_test_set.shape

In [ ]:
penguins_train_X = penguins_train_set.drop('species',axis=1)
penguins_train_y = penguins_train_set['species'].copy()
penguins_test_X = penguins_test_set.drop('species',axis=1)
penguins_test_y = penguins_test_set[['species']].copy()

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
import numpy as np
num_attribs = list(penguins_train_X.select_dtypes(include=[np.number]))
cat_attribs = list(penguins_train_X.select_dtypes(exclude=[np.number]))

full_pipeline = ColumnTransformer([
        ("num", StandardScaler(), num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),
    ])

penguins_train_X_prepared = full_pipeline.fit_transform(penguins_train_X)
penguins_test_X_prepared = full_pipeline.transform(penguins_test_X)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [ ]:
# from https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html
names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes"]

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB(),
    ]

In [ ]:
X_train = penguins_train_X_prepared
y_train = penguins_train_y
X_test = penguins_test_X_prepared
y_test = penguins_test_y

for name, clf in zip(names, classifiers):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    # evaluate predictions
    accuracy = clf.score(X_test, y_test)
    print("%s Accuracy: %.2f%%" % (name,accuracy * 100.0))

Let's define a classifier that always classifies a species as "not Chinstrap":

In [ ]:
from sklearn.base import BaseEstimator

class NeverChinstrapClassifier(BaseEstimator):
    def fit(self, X, y=None):
        return self
    def predict(self, X):
        return np.zeros((len(X), 1), dtype=bool)

In [ ]:
y_train_chinstrap = penguins_train_y == 'Chinstrap'

In [ ]:
y_train_chinstrap.sum()

In [ ]:
from sklearn.model_selection import cross_val_score
never_chinstrap_clf = NeverChinstrapClassifier()
cvs = cross_val_score(never_chinstrap_clf,penguins_train_X_prepared,y_train_chinstrap,cv=5,scoring='accuracy')
cvs

In [ ]:
np.mean(cvs)

Let's take a look at some additional ways to assess how good a model (classifier) is:

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
forest_clf = DecisionTreeClassifier()

In [ ]:
hasattr(forest_clf, "predict_proba")

In [ ]:
hasattr(forest_clf, "decision_function")

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict
forest_clf = RandomForestClassifier(n_estimators=100,max_depth=3,random_state=42)

y_probas_forest = cross_val_predict(forest_clf, penguins_train_X_prepared, y_train_chinstrap, cv=3,
                                    method="predict_proba")

In [ ]:
y_train_pred = cross_val_predict(forest_clf, penguins_train_X_prepared, y_train_chinstrap, cv=3)

In [ ]:
confusion_matrix(y_train_pred,y_train_chinstrap)

In [ ]:
y_probas_forest

In [ ]:
from sklearn.metrics import roc_curve
y_scores_forest = y_probas_forest[:, 1]   # score = proba of positive class
fpr_forest, tpr_forest, thresholds_forest = roc_curve(y_train_chinstrap,y_scores_forest)#,pos_label='Male')

In [ ]:
from sklearn.metrics import precision_recall_curve

precisions, recalls, thresholds = precision_recall_curve(y_train_chinstrap,y_scores_forest)

In [ ]:
def plot_precision_recall_vs_threshold(precisions, recalls, thresholds):
    plt.plot(thresholds, precisions[:-1], "b--", label="Precision", linewidth=2)
    plt.plot(thresholds, recalls[:-1], "g-", label="Recall", linewidth=2)
    plt.legend(loc="center right", fontsize=16) # Not shown in the book
    plt.xlabel("Threshold", fontsize=16)        # Not shown
    plt.grid(True)                              # Not shown

plt.figure(figsize=(8, 4))                                                                  # Not shown
plot_precision_recall_vs_threshold(precisions, recalls, thresholds)
plt.show()

In [ ]:
def plot_precision_vs_recall(precisions, recalls):
    plt.plot(recalls, precisions, "b-", linewidth=2)
    plt.xlabel("Recall", fontsize=16)
    plt.ylabel("Precision", fontsize=16)
    plt.axis([0, 1, 0, 1])
    plt.grid(True)

plt.figure(figsize=(8, 6))
plot_precision_vs_recall(precisions, recalls)

In [ ]:
threshold_90_precision = thresholds[np.argmax(precisions >= 0.90)] 

In [ ]:
threshold_90_precision

In [ ]:
y_train_pred_90 = (y_scores_forest >= threshold_90_precision)

In [ ]:
from sklearn.metrics import precision_score, recall_score
print(precision_score(y_train_chinstrap, y_train_pred_90))
print(recall_score(y_train_chinstrap, y_train_pred_90))

In [ ]:
def plot_roc_curve(fpr, tpr, label=None):
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'k--') # dashed diagonal
    plt.axis([0, 1, 0, 1])                                    # Not shown in the book
    plt.xlabel('False Positive Rate (Fall-Out)', fontsize=14) # Not shown
    plt.ylabel('True Positive Rate (Recall)', fontsize=14)    # Not shown
    plt.grid(True)                                            # Not shown

plot_roc_curve(fpr_forest, tpr_forest)
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_train_chinstrap, y_scores_forest)

## Note:
(from https://machinelearningmastery.com/roc-curves-and-precision-recall-curves-for-classification-in-python/)

ROC curves should be used when there are roughly equal numbers of observations for each class.

Precision-Recall curves should be used when there is a moderate to large class imbalance.
    